In [16]:
import sys

In [17]:
sys.version

'3.8.10 (default, Jan  4 2024, 11:59:19) \n[GCC 11.4.0]'

# Configuration

In [46]:
import numpy as np
import os

from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

In [47]:

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
  except RuntimeError as e:
    # Visible devices must be set at program startup
    print(e)

In [48]:
gpus

[]

In [20]:
spec = model_spec.get('efficientdet_lite1')

In [21]:
object_detector.DataLoader

tensorflow_examples.lite.model_maker.core.data_util.object_detector_dataloader.DataLoader

In [22]:
object_detector.DataLoader.from_pascal_voc?

Signature:
object_detector.DataLoader.from_pascal_voc(
    images_dir: str,
    annotations_dir: str,
    label_map: Union[List[str], Dict[int, str], str],
    annotation_filenames: Union[Collection[str], NoneType] = None,
    ignore_difficult_instances: bool = False,
    num_shards: int = 100,
    max_num_images: Union[int, NoneType] = None,
    cache_dir: Union[str, NoneType] = None,
    cache_prefix_filename: Union[str, NoneType] = None,
) -> ~DetectorDataLoader
Docstring:
Loads from dataset with PASCAL VOC format.

Refer to
https://towardsdatascience.com/coco-data-format-for-object-detection-a4c5eaf518c5
for the description of PASCAL VOC data format.

LabelImg Tool (https://github.com/tzutalin/labelImg) can annotate the image
and save annotations as XML files in PASCAL VOC data format.

Annotations are in the folder: `annotations_dir`.
Raw images are in the foloder: `images_dir`.

Args:
  images_dir: Path to directory that store raw images.
  annotations_dir: Path to the annotation

In [23]:
# object_detector.DataLoader.from_csv?

# Prepare the dataset

In [24]:
import os

In [25]:
os.getcwd()

'/home/technonatura-ai/Desktop/R2045/CenterStage-WorldChampionship-TFLite'

In [36]:
train_data = object_detector.DataLoader.from_pascal_voc(f"{os.getcwd()}/PascalVOC-CenterCrop/train/imgs", f"{os.getcwd()}/PascalVOC-CenterCrop/train/annotations",["teamprop_red", "teamprop_blue"])
validation_data  = object_detector.DataLoader.from_pascal_voc(f"{os.getcwd()}/PascalVOC-CenterCrop/valid/imgs", f"{os.getcwd()}/PascalVOC-CenterCrop/valid/annotations", [ "teamprop_red", "teamprop_blue"])

In [37]:
train_data.label_map

{1: 'teamprop_red', 2: 'teamprop_blue'}

In [40]:
test_data  = object_detector.DataLoader.from_pascal_voc(f"{os.getcwd()}/PascalVOC-CenterCrop/test/imgs", f"{os.getcwd()}/PascalVOC-CenterCrop/test/annotations", [ "teamprop_red", "teamprop_blue"])

In [43]:
len(train_data)

789

In [41]:
len(validation_data)

222

In [42]:
len(test_data)

128

# Train the Model

In [45]:
gpus

[]

In [44]:
spec0 = model_spec.get('efficientdet_lite0')
model = object_detector.create(train_data, model_spec=spec0, batch_size=2, train_whole_model=True, validation_data=validation_data)


Epoch 1/50


2024-03-05 15:12:22.257771: W tensorflow/core/framework/dataset.cc:768] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


394/394 [==============================] - ETA: 0s - det_loss: 0.6473 - cls_loss: 0.3804 - box_loss: 0.0053 - reg_l2_loss: 0.0634 - loss: 0.7107 - learning_rate: 0.0053 - gradient_norm: 4.6021

2024-03-05 15:13:12.643356: W tensorflow/core/framework/dataset.cc:768] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


394/394 [==============================] - 53s 113ms/step - det_loss: 0.6463 - cls_loss: 0.3799 - box_loss: 0.0053 - reg_l2_loss: 0.0634 - loss: 0.7096 - learning_rate: 0.0053 - gradient_norm: 4.5976 - val_det_loss: 0.2302 - val_cls_loss: 0.1616 - val_box_loss: 0.0014 - val_reg_l2_loss: 0.0636 - val_loss: 0.2938
Epoch 2/50
394/394 [==============================] - ETA: 0s - det_loss: 0.3040 - cls_loss: 0.1902 - box_loss: 0.0023 - reg_l2_loss: 0.0636 - loss: 0.3676 - learning_rate: 0.0025 - gradient_norm: 3.4232

...

2024-03-05 15:21:18.586661: W tensorflow/core/framework/dataset.cc:768] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


394/394 [==============================] - 43s 110ms/step - det_loss: 0.1756 - cls_loss: 0.1198 - box_loss: 0.0011 - reg_l2_loss: 0.0637 - loss: 0.2393 - learning_rate: 0.0022 - gradient_norm: 2.8179 - val_det_loss: 0.1113 - val_cls_loss: 0.0940 - val_box_loss: 3.4706e-04 - val_reg_l2_loss: 0.0637 - val_loss: 0.1751
Epoch 13/50
394/394 [==============================] - ETA: 0s - det_loss: 0.1743 - cls_loss: 0.1177 - box_loss: 0.0011 - reg_l2_loss: 0.0637 - loss: 0.2381 - learning_rate: 0.0021 - gradient_norm: 2.8943

2024-03-05 15:22:02.194498: W tensorflow/core/framework/dataset.cc:768] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


394/394 [==============================] - 44s 111ms/step - det_loss: 0.1742 - cls_loss: 0.1176 - box_loss: 0.0011 - reg_l2_loss: 0.0637 - loss: 0.2380 - learning_rate: 0.0021 - gradient_norm: 2.8916 - val_det_loss: 0.1275 - val_cls_loss: 0.1092 - val_box_loss: 3.6664e-04 - val_reg_l2_loss: 0.0637 - val_loss: 0.1913
Epoch 14/50
271/394 [===================>..........] - ETA: 13s - det_loss: 0.1643 - cls_loss: 0.1163 - box_loss: 9.6085e-04 - reg_l2_loss: 0.0637 - loss: 0.2281 - learning_rate: 0.0021 - gradient_norm: 2.8195

KeyboardInterrupt: 

In [25]:
Values = model.evaluate(test_data)

1/1 [==============================] - 2s 2s/step



In [26]:
print(Values)

{'AP': 0.9186616, 'AP50': 1.0, 'AP75': 1.0, 'APs': -1.0, 'APm': 0.9141914, 'APl': 0.9245618, 'ARmax1': 0.92583334, 'ARmax10': 0.9546667, 'ARmax100': 0.9546667, 'ARs': -1.0, 'ARm': 0.915, 'ARl': 0.9628146, 'AP_/teamprop_red': 0.88633716, 'AP_/teamprop_blue': 0.950986}


In [16]:
spec4 = model_spec.get('efficientdet_lite4')

In [17]:

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [22]:

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
  except RuntimeError as e:
    # Visible devices must be set at program startup
    print(e)

In [19]:
model2 = None

In [15]:
spec4 = model_spec.get('efficientdet_lite1')

In [16]:
model2 = object_detector.create(train_data, model_spec=spec4, batch_size=12, epochs=50, train_whole_model=True, validation_data=validation_data)


Epoch 1/50


2024-01-04 18:47:12.087322: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8801
2024-01-04 18:47:12.814098: W tensorflow/stream_executor/gpu/asm_compiler.cc:230] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.9
2024-01-04 18:47:12.814134: W tensorflow/stream_executor/gpu/asm_compiler.cc:233] Used ptxas at ptxas
2024-01-04 18:47:12.814158: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


29/29 [==============================] - 20s 214ms/step - det_loss: 1.3681 - cls_loss: 0.9662 - box_loss: 0.0080 - reg_l2_loss: 0.0706 - loss: 1.4387 - learning_rate: 0.0115 - gradient_norm: 1.8255 - val_det_loss: 1.0233 - val_cls_loss: 0.8133 - val_box_loss: 0.0042 - val_reg_l2_loss: 0.0706 - val_loss: 1.0939
Epoch 2/50
29/29 [==============================] - 5s 181ms/step - det_loss: 0.4981 - cls_loss: 0.3532 - box_loss: 0.0029 - reg_l2_loss: 0.0706 - loss: 0.5688 - learning_rate: 0.0150 - gradient_norm: 1.8802 - val_det_loss: 0.3812 - val_cls_loss: 0.2686 - val_box_loss: 0.0023 - val_reg_l2_loss: 0.0706 - val_loss: 0.4519
Epoch 3/50
29/29 [==============================] - 5s 180ms/step - det_loss: 0.2676 - cls_loss: 0.1873 - box_loss: 0.0016 - reg_l2_loss: 0.0707 - loss: 0.3383 - learning_rate: 0.0149 - gradient_norm: 1.3890 - val_det_loss: 0.3123 - val_cls_loss: 0.2061 - val_box_loss: 0.0021 - val_reg_l2_loss: 0.0707 - val_loss: 0.3829
Epoch 4/50
29/29 [==========================

In [17]:
model2.export(export_dir='.',     tflite_filename='model_efficientDet1.tflite',
    label_filename='labels_efficientDet1.txt',
    vocab_filename='vocab_efficientDet1.txt',
    saved_model_filename='saved_model',
    tfjs_folder_name='tfjs')

2024-01-04 18:51:47.776763: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2024-01-04 18:51:58.450380: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'resample_p7/PartitionedCall' has 1 outputs but the _output_shapes attribute specifies shapes for 3 outputs. Output shapes may be inaccurate.
2024-01-04 18:52:01.592993: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2024-01-04 18:52:01.593014: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2024-01-04 18:52:01.593592: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmp011nx5tm
2024-01-04 18:52:01.653132: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2024-01-04 18:52:01.653155: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if pre

In [ ]:
spec4 = model_spec.get('efficientdet_lite1')

In [18]:
Values2 = model2.evaluate(test_data)

1/1 [==============================] - 2s 2s/step



In [19]:
print(Values2)

{'AP': 0.9544427, 'AP50': 1.0, 'AP75': 1.0, 'APs': -1.0, 'APm': 0.91936195, 'APl': 0.96342546, 'ARmax1': 0.95883334, 'ARmax10': 0.9751667, 'ARmax100': 0.9751667, 'ARs': -1.0, 'ARm': 0.935, 'ARl': 0.9820366, 'AP_/teamprop_red': 0.9324126, 'AP_/teamprop_blue': 0.9764728}


# visualise the model

In [27]:

import cv2

from PIL import Image

model_path = 'model.tflite'

# Load the labels into a list
classes = ['???'] * model.model_spec.config.num_classes
label_map = model.model_spec.config.label_map

NameError: name 'model' is not defined

In [ ]:
for label_id, label_name in label_map.as_dict().items():
  classes[label_id-1] = label_name

# Define a list of colors for visualization
COLORS = np.random.randint(0, 255, size=(len(classes), 3), dtype=np.uint8)

def preprocess_image(image_path, input_size):
  """Preprocess the input image to feed to the TFLite model"""
  img = tf.io.read_file(image_path)
  img = tf.io.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.uint8)
  original_image = img
  resized_img = tf.image.resize(img, input_size)
  resized_img = resized_img[tf.newaxis, :]
  resized_img = tf.cast(resized_img, dtype=tf.uint8)
  return resized_img, original_image


def detect_objects(interpreter, image, threshold):
  """Returns a list of detection results, each a dictionary of object info."""

  signature_fn = interpreter.get_signature_runner()

  # Feed the input image to the model
  output = signature_fn(images=image)

  # Get all outputs from the model
  count = int(np.squeeze(output['output_0']))
  scores = np.squeeze(output['output_1'])
  classes = np.squeeze(output['output_2'])
  boxes = np.squeeze(output['output_3'])

  results = []
  for i in range(count):
    if scores[i] >= threshold:
      result = {
        'bounding_box': boxes[i],
        'class_id': classes[i],
        'score': scores[i]
      }
      results.append(result)
  return results


def run_odt_and_draw_results(image_path, interpreter, threshold=0.5):
  """Run object detection on the input image and draw the detection results"""
  # Load the input shape required by the model
  _, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']

  # Load the input image and preprocess it
  preprocessed_image, original_image = preprocess_image(
      image_path,
      (input_height, input_width)
    )

  # Run object detection on the input image
  results = detect_objects(interpreter, preprocessed_image, threshold=threshold)

  # Plot the detection results on the input image
  original_image_np = original_image.numpy().astype(np.uint8)
  for obj in results:
    # Convert the object bounding box from relative coordinates to absolute
    # coordinates based on the original image resolution
    ymin, xmin, ymax, xmax = obj['bounding_box']
    xmin = int(xmin * original_image_np.shape[1])
    xmax = int(xmax * original_image_np.shape[1])
    ymin = int(ymin * original_image_np.shape[0])
    ymax = int(ymax * original_image_np.shape[0])

    # Find the class index of the current object
    class_id = int(obj['class_id'])

    # Draw the bounding box and label on the image
    color = [int(c) for c in COLORS[class_id]]
    cv2.rectangle(original_image_np, (xmin, ymin), (xmax, ymax), color, 2)
    # Make adjustments to make the label visible for all objects
    y = ymin - 15 if ymin - 15 > 15 else ymin + 15
    label = "{}: {:.0f}%".format(classes[class_id], obj['score'] * 100)
    cv2.putText(original_image_np, label, (xmin, y),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

  # Return the final image
  original_uint8 = original_image_np.astype(np.uint8)
  return original_uint8

In [ ]:
#@title Run object detection and show the detection results

INPUT_IMAGE_URL = "https://storage.googleapis.com/cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg" #@param {type:"string"}
DETECTION_THRESHOLD = 0.3 #@param {type:"number"}

TEMP_FILE = '/tmp/image.png'

im = Image.open(TEMP_FILE)
im.thumbnail((512, 512), Image.ANTIALIAS)
im.save(TEMP_FILE, 'PNG')

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Run inference and draw detection result on the local copy of the original file
detection_result_image = run_odt_and_draw_results(
    TEMP_FILE,
    interpreter,
    threshold=DETECTION_THRESHOLD
)

# Show the detection result
Image.fromarray(detection_result_image)

In [ ]:
tf.lite.Interpreter?

# Evaluate the model

In [31]:
model.evaluate(test_data)

1/1 [==============================] - 2s 2s/step



{'AP': 0.9186616,
 'AP50': 1.0,
 'AP75': 1.0,
 'APs': -1.0,
 'APm': 0.9141914,
 'APl': 0.9245618,
 'ARmax1': 0.92583334,
 'ARmax10': 0.9546667,
 'ARmax100': 0.9546667,
 'ARs': -1.0,
 'ARm': 0.915,
 'ARl': 0.9628146,
 'AP_/teamprop_red': 0.88633716,
 'AP_/teamprop_blue': 0.950986}

In [32]:
model.export(export_dir='.')

2024-01-04 17:13:35.849422: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2024-01-04 17:13:44.630864: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'resample_p7/PartitionedCall' has 1 outputs but the _output_shapes attribute specifies shapes for 3 outputs. Output shapes may be inaccurate.
2024-01-04 17:13:47.347961: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2024-01-04 17:13:47.347983: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2024-01-04 17:13:47.348414: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpwaq4uur4
2024-01-04 17:13:47.395477: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2024-01-04 17:13:47.395496: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if pre

In [23]:
model2.export(export_dir='.',     tflite_filename='model_efficientDet4.tflite',
    label_filename='labels_efficientDet4.txt',
    vocab_filename='vocab_efficientDet4.txt',
    saved_model_filename='saved_model',
    tfjs_folder_name='tfjs')

2024-01-04 18:04:36.785506: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2024-01-04 18:04:52.149373: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'resample_p7/PartitionedCall' has 1 outputs but the _output_shapes attribute specifies shapes for 3 outputs. Output shapes may be inaccurate.
2024-01-04 18:04:57.466311: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2024-01-04 18:04:57.466335: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2024-01-04 18:04:57.468407: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpmuxpr_pn
2024-01-04 18:04:57.553791: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2024-01-04 18:04:57.553820: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if pre

In [36]:
model.evaluate_tflite('model.tflite', test_data)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


49/49 [==============================] - 43s 876ms/step



{'AP': 0.9082702,
 'AP50': 1.0,
 'AP75': 1.0,
 'APs': -1.0,
 'APm': 0.8655941,
 'APl': 0.91969395,
 'ARmax1': 0.92583334,
 'ARmax10': 0.92583334,
 'ARmax100': 0.92583334,
 'ARs': -1.0,
 'ARm': 0.87,
 'ARl': 0.93524027,
 'AP_/teamprop_red': 0.87334126,
 'AP_/teamprop_blue': 0.94319904}

In [24]:
model2.evaluate_tflite('model_efficientDet4.tflite', test_data)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


 3/49 [>.............................] - ETA: 14:16

KeyboardInterrupt: 